In [1]:
import os
from glob import glob
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *
from mitequinox.plot import *

ModuleNotFoundError: No module named 'mitequinox'

In [ ]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster()
w = cluster.scale(jobs=1)

In [ ]:
from dask.distributed import Client
client = Client(cluster)

In [ ]:
client